### Import

In [165]:
import os
import pickle
import argparse
import numpy as np
import pandas as pd
import time

from tqdm import tqdm 
from collections import defaultdict

# https://github.com/ZFTurbo/Weighted-Boxes-Fusion
# pip install ensemble_boxes
from ensemble_boxes import * 

In [166]:
ensemble_output_dir = '/mmdetection/inference/ensembles/fast-rcnn_r50-KFold'

ensemble_output_list = os.listdir(ensemble_output_dir)
print(ensemble_output_list)

['faster-rcnn_r50_fpn_Fold3_submission.csv', 'faster-rcnn_r50_fpn_Fold4_submission.csv', 'faster-rcnn_r50_fpn_Fold5_submission.csv', 'faster-rcnn_r50_fpn_Fold1_submission.csv', 'faster-rcnn_r50_fpn_Fold2_submission.csv']


In [167]:
fold_pd_list = []
for idx, output in enumerate(ensemble_output_list):
    with open(os.path.join(ensemble_output_dir, output), 'rb') as f:
        data = pd.read_csv(f)
    print('Null Index : ', data[data['PredictionString'].isnull()].index)
    fold_pd_list.append(data)

Null Index :  Int64Index([], dtype='int64')
Null Index :  Int64Index([], dtype='int64')
Null Index :  Int64Index([], dtype='int64')
Null Index :  Int64Index([], dtype='int64')
Null Index :  Int64Index([], dtype='int64')


In [168]:
"""
{ "fold1": { "train/0000.jpg": { "bboxes": [], "labels": [], "scores": [], } "train/0001.jpg": { ... } ... }, 
  "fold1": { "train/0000.jpg": { "bboxes": [], "labels": [], "scores": [], } "train/0001.jpg": { ... } ... }, ...}
"""
iou_thr = 0.5
skip_box_thr = 0.0001
weights = [1] * len(ensemble_output_list)

csv_predictions = []
for image_id in fold_pd_list[0]['image_id']:
    labels_list, scores_list, bboxes_list = [], [], []
        
    for fold_pd in fold_pd_list:
        labels, scores, bboxes = [], [], []
        
        predict = list(fold_pd[fold_pd['image_id']==image_id]['PredictionString'])[0]
        predict = predict.strip().split(' ')
        predict_list = [list(map(float, predict[i:i+6])) for i in range(0, len(predict), 6)] # 6개 단위로 묶기
        
        for predict_ in predict_list: # 한 이미지 내 label, score, bbox
            label = predict_[0]
            score = predict_[1]
            bbox  = [predict_[2]/1024., predict_[3]/1024., predict_[4]/1024., predict_[5]/1024.]
            
            labels.append(label)
            scores.append(score)
            bboxes.append(bbox)

        labels_list.append(labels)
        scores_list.append(scores)
        bboxes_list.append(bboxes)

    # nms, soft-nms, weighted_boxes_fusion
    bboxes, scores, labels = weighted_boxes_fusion(bboxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    predict = ''
    for infos in zip(labels, scores, bboxes):
        infos = list(infos)
        
        predict += str(int(infos[0])) + ' '
        predict += str(infos[1]) + ' '
        bbox = infos[2].tolist()
        predict += str(bbox[0]*1024.) + ' ' + str(bbox[1]*1024.) + ' ' + str(bbox[2]*1024.) + ' ' + str(bbox[3]*1024.) + ' '

    csv_predictions.append(predict)

In [169]:
submission = pd.DataFrame()
submission['PredictionString'] = csv_predictions
submission['image_id'] = fold_pd_list[0]['image_id']

submission.to_csv(f'/mmdetection/inference/submission/ensemble_submission.csv', index=None)
submission.head()

,PredictionString,image_id
0,7 0.9982357025146484 599.6793212890625 521.624...,test/0000.jpg
1,5 0.6400420665740967 343.70184326171875 244.21...,test/0001.jpg
2,1 0.8847388029098511 258.0965270996094 309.739...,test/0002.jpg
3,9 0.8334046006202698 78.46959686279297 153.520...,test/0003.jpg
4,0 0.41900140047073364 185.37600708007812 322.9...,test/0004.jpg
